In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
c_train=pd.read_csv('C:/Users/Hp/OneDrive/Documents/IIT KANPUR PYTHON/6th PROJECT/train.csv')

In [3]:
c_test=pd.read_csv('C:/Users/Hp/OneDrive/Documents/IIT KANPUR PYTHON/6th PROJECT/test_share.csv')

In [4]:
do=pd.read_csv('C:/Users/Hp/OneDrive/Documents/IIT KANPUR PYTHON/6th PROJECT/demographic_details.csv')

In [5]:
med=pd.read_csv('C:/Users/Hp/OneDrive/Documents/IIT KANPUR PYTHON/6th PROJECT/medical_history.csv')

In [6]:
com=pd.merge(do,med,how='outer',on='PatientId')

In [7]:
train=pd.merge(c_train,com,how='left',on='PatientId')
test=pd.merge(c_test,com,how='left',on='PatientId')

In [8]:
train.head()

,PatientId,AppointmentID,ScheduledDay,AppointmentDay,SMS_received,No-show,Gender,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap
0,2.987250e+13,5642903,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,0,No,F,62,JARDIM DA PENHA,0,1,0,0,0
1,5.589978e+14,5642503,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,0,No,M,56,JARDIM DA PENHA,0,0,0,0,0
2,8.679512e+11,5642828,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,0,No,F,8,PONTAL DE CAMBURI,0,0,0,0,0
3,8.841186e+12,5642494,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,0,No,F,56,JARDIM DA PENHA,0,1,1,0,0
4,9.598513e+13,5626772,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,0,No,F,76,REPÚBLICA,0,1,0,0,0


In [9]:
train['Neighbourhood'].value_counts()

JARDIM CAMBURI                 6948
MARIA ORTIZ                    5203
RESISTÊNCIA                    3960
JARDIM DA PENHA                3509
ITARARÉ                        3171
                               ... 
ILHA DO BOI                      31
ILHA DO FRADE                    10
AEROPORTO                         7
ILHAS OCEÂNICAS DE TRINDADE       2
PARQUE INDUSTRIAL                 1
Name: Neighbourhood, Length: 81, dtype: int64

In [10]:
drop_cols=['PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay']

In [11]:
train.drop(drop_cols,1,inplace=True)
test.drop(drop_cols,1,inplace=True)

In [12]:
train['data']='train'
test['data']='test'

all_data=pd.concat([train,test],0,sort=False)

In [13]:
all_data['gender_f']=(all_data['Gender']=='F').astype(int)
del all_data['Gender']

In [14]:
all_data.shape

(110344, 11)

In [15]:
k=all_data['Neighbourhood'].value_counts()
cats=k[k>2000].index

In [16]:
for cat in cats:
    name='Neighbourhood_'+cat
    all_data[name]=(all_data['Neighbourhood']==cat).astype(int)
del all_data['Neighbourhood']

In [17]:
all_data.shape

(110344, 31)

In [18]:
all_data.head(2)

,SMS_received,No-show,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,data,gender_f,...,Neighbourhood_SANTO ANDRÉ,Neighbourhood_CARATOÍRA,Neighbourhood_JABOUR,Neighbourhood_SÃO PEDRO,Neighbourhood_ILHA DO PRÍNCIPE,Neighbourhood_NOVA PALESTINA,Neighbourhood_DA PENHA,Neighbourhood_ANDORINHAS,Neighbourhood_ROMÃO,Neighbourhood_GURIGICA
0,0,No,62,0,1,0,0,0,train,1,...,0,0,0,0,0,0,0,0,0,0
1,0,No,56,0,0,0,0,0,train,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
target='No-show'

In [20]:
x_tarin=all_data.drop([target,'data'],1)[all_data['data']=='train']
y_train=all_data[target][all_data['data']=='train']
x_test=all_data.drop([target,'data'],1)[all_data['data']=='test']

# Model Fitting

In [21]:
clf=GradientBoostingClassifier()

In [22]:
gbm_params={'n_estimators':[50,100,200,500,700],
           'learning_rate': [0.01,.05,0.1,0.4,0.8,1],
            'max_depth':[1,2,3,4,5,6],
            'min_samples_split':[2,5,10,20],
            'min_samples_leaf':[2,5,10,20],
            'subsample':[0.5,0.8,1],
            'max_features':[5,10,15,20,30,45,55,65]
           }

In [23]:
ramdom_search=RandomizedSearchCV(clf,scoring='roc_auc',param_distributions=gbm_params,cv=10,n_iter=10, n_jobs=-1,verbose=20)

In [24]:
ramdom_search.fit(x_tarin,y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.58274264        nan 0.58835965 0.58071654        nan        nan
        nan 0.58199636        nan        nan]
  warnings.warn(


RandomizedSearchCV(cv=10, estimator=GradientBoostingClassifier(), n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1, 0.4,
                                                          0.8, 1],
                                        'max_depth': [1, 2, 3, 4, 5, 6],
                                        'max_features': [5, 10, 15, 20, 30, 45,
                                                         55, 65],
                                        'min_samples_leaf': [2, 5, 10, 20],
                                        'min_samples_split': [2, 5, 10, 20],
                                        'n_estimators': [50, 100, 200, 500,
                                                         700],
                                        'subsample': [0.5, 0.8, 1]},
                   scoring='roc_auc', verbose=20)

In [25]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.6f} (std: {1:.6f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [26]:
report(ramdom_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.588360 (std: 0.041170)
Parameters: {'subsample': 1, 'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 20, 'max_features': 15, 'max_depth': 4, 'learning_rate': 0.01}

Model with rank: 2
Mean validation score: 0.582743 (std: 0.043708)
Parameters: {'subsample': 0.8, 'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 10, 'max_features': 20, 'max_depth': 5, 'learning_rate': 0.05}

Model with rank: 3
Mean validation score: 0.581996 (std: 0.046958)
Parameters: {'subsample': 0.5, 'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 5, 'max_depth': 5, 'learning_rate': 0.1}

Model with rank: 4
Mean validation score: 0.580717 (std: 0.044485)
Parameters: {'subsample': 0.5, 'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 10, 'max_depth': 2, 'learning_rate': 0.8}

Model with rank: 5
Mean validation score: nan (std: nan)
Parameters: {'subsample': 1, 'n_estimators'

In [27]:
rs=GradientBoostingClassifier(**{'subsample': 0.8, 'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 10, 'max_features': 20, 'max_depth': 5, 'learning_rate': 0.05})

In [28]:
rs.fit(x_tarin,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_depth=5, max_features=20,
                           min_samples_leaf=10, min_samples_split=5,
                           n_estimators=500, subsample=0.8)

In [29]:
submissions=pd.DataFrame({'Junk':rs.predict_proba(x_test)[:,1]})

In [30]:
submissions.to_csv('sample_submission.csv',index=False)